In [4]:
import numpy as np
import cv2
import tensorflow as tf
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive')
import numpy as np
import cv2
import argparse
from imutils.perspective import four_point_transform
from skimage.segmentation import clear_border
import numpy as np
import imutils
import cv2
# Load the trained model
model = tf.keras.models.load_model('/content/model-OCR.h5')
import numpy as np

def get_individual_cell(puzzle, row, col):
    # Determinining size of each cell
    puzzle_height, puzzle_width = puzzle.shape[:2]
    cell_height, cell_width = puzzle_height // 9, puzzle_width // 9

    # Calculate the starting and ending coordinates of the cell
    start_row, end_row = row * cell_height, (row + 1) * cell_height
    start_col, end_col = col * cell_width, (col + 1) * cell_width

    # Extract the individual cell from the puzzle
    cell = puzzle[start_row:end_row, start_col:end_col]

    return cell



def preprocess_cell(cell):
    # Resize the cell to match the model input size
    cell = cv2.resize(cell, (48, 48))
    # Normalize pixel values
    cell = cell.reshape((1, 48, 48, 1)).astype('float32') / 255
    return cell

def recognize_digit_with_threshold(cell, confidence_threshold=0.5):
    processed_cell = preprocess_cell(cell)
    predictions = model.predict(processed_cell)



    # Check if the model is confident enough in its prediction
    max_confidence = np.max(predictions)
    if max_confidence >= confidence_threshold:
        digit_prediction = np.argmax(predictions, axis=-1)
        return digit_prediction
    else:
        return None

def is_valid_move(board, row, col, num):
    # Check if 'num' is not present in the current row, column, and 3x3 grid
    return (
        num not in board[row, :] and
        num not in board[:, col] and
        num not in board[row // 3 * 3: (row // 3 + 1) * 3, col // 3 * 3: (col // 3 + 1) * 3]
    )

def find_empty_location(board):
    # Find the first empty cell (denoted by 0)
    for i in range(9):
        for j in range(9):
            if board[i, j] == 0:
                return i, j
    return None

def solve_sudoku(board):
    # Find the first empty cell
    empty_location = find_empty_location(board)

    # If there are no empty cells, the puzzle is solved
    if empty_location is None:
        return True

    row, col = empty_location

    # Try filling the empty cell with digits 1 to 9
    for num in range(1, 10):
        if is_valid_move(board, row, col, num):
            # Make the move
            board[row, col] = num

            # Recursively try to solve the rest of the puzzle
            if solve_sudoku(board):
                return True

            # If the current move leads to an invalid solution, backtrack
            board[row, col] = 0

    # If no digit works for the current empty cell, backtrack
    return False

def find_puzzle(image, debug=False):
	gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	blurred = cv2.GaussianBlur(gray, (7, 7), 3)
	thresh = cv2.adaptiveThreshold(blurred, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
	thresh = cv2.bitwise_not(thresh)
	# check to see if we are visualizing each step of the image
	# processing pipeline (in this case, thresholding)

  # find contours in the thresholded image and sort them by size in
	# descending order
	cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
	# initialize a contour that corresponds to the puzzle outline
	puzzleCnt = None
	# loop over the contours
	for c in cnts:
		# approximate the contour
		peri = cv2.arcLength(c, True)
		approx = cv2.approxPolyDP(c, 0.02 * peri, True)
		# if our approximated contour has four points, then we can
		# assume we have found the outline of the puzzle
		if len(approx) == 4:
			puzzleCnt = approx
			break
    # if the puzzle contour is empty then our script could not find
	# the outline of the Sudoku puzzle so raise an error
	if puzzleCnt is None:
		raise Exception(("Could not find Sudoku puzzle outline. "
			"Try debugging your thresholding and contour steps."))
  # apply a four point perspective transform to both the original
	# image and grayscale image to obtain a top-down bird's eye view
	# of the puzzle
	puzzle = four_point_transform(image, puzzleCnt.reshape(4, 2))
	warped = four_point_transform(gray, puzzleCnt.reshape(4, 2))
	# return a 2-tuple of puzzle in both RGB and grayscale
	return (puzzle, warped)

# Load the puzzle image
image = cv2.imread('/content/drive/MyDrive/Screenshot 2024-01-09 234449.png')

# Find and extract the puzzle from the image
puzzle, warped = find_puzzle(image, debug=True)

sudoku_grid = np.zeros((9, 9), dtype=int)
for row in range(9):
    for col in range(9):
        cell = get_individual_cell(warped, row, col)

        # Recognize digit with confidence threshold
        digit = recognize_digit_with_threshold(cell)

        # If the model is confident, assign the digit to the grid
        if digit is not None:
            sudoku_grid[row, col] = digit

# Solve Sudoku using backtracking
if solve_sudoku(sudoku_grid):
    print("Solved Sudoku:")
    print(sudoku_grid)
else:
    print("No solution found.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1/1 [==============================] - 0s 25ms/step
Solved Sudoku:
[[2 6 7 3 5 1 4 8 9]
 [9 1 5 8 2 4 6 7 3]
 [8 4 3 6 7 9 2 1 5]
 [1 8 2 9 6 3 7 5 4]
 [5 7 6 1 4 2 3 9 8]
 [3 9 4 5 8 7 1 6 2]
 [4 2 1 7 9 8 5 3 6]
 [7 5 8 4 3 6 9 2 1]
 [6 3 9 2 1 5 8 4 7]]
